In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import keras

from keras.utils import np_utils
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from tensorflow.keras.layers import BatchNormalization
import os
print(os.listdir(r"C:\Users\thanawin\Desktop\Deep-learing\90_10_10\detect_split\train"))

['anger', 'disgust', 'fear', 'hapiness', 'neutral', 'sad', 'surprise']


In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.datasets import make_classification
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [3]:
data_path = r'C:\Users\thanawin\Desktop\Deep-learing\90_10_10\detect_split\train'
data_dir_list = os.listdir(data_path)

num_epoch=10

img_data_list=[]


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        #input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

Loaded the images of dataset-anger

Loaded the images of dataset-disgust

Loaded the images of dataset-fear

Loaded the images of dataset-hapiness

Loaded the images of dataset-neutral

Loaded the images of dataset-sad

Loaded the images of dataset-surprise



(20451, 128, 128, 3)

In [4]:
num_classes = 7

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:3896]=0 #3897 0 - 3896
labels[3897:6435]=1 #2538 6435
labels[6436:8752]=2 #2317 8752
labels[8753:11830]=3 #3078 11830
labels[11831:14307]=4 #2477 14307
labels[14308:16991]=5 #2684 16991
labels[16992:20451]=6 #3460 20451

names = ['
$ ls
0/  1/  2/  3/  4/  5/  6/

','disgust','fear','hapiness','neutral','sad','surprise']

def getLabel(id):
    return ['anger','disgust','fear','hapiness','neutral','sad','surprise'][id]

In [5]:
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=2)
x_test=X_test

In [6]:
#CNN
def create_model():
        model = Sequential() #Sequential model allows to build a model layer by layer

        #--------------------------------Input layer------------------------------------------# 
        #COnvolution layer parametrs:
        #No of output filters: 32
        #kernel_size: specifying the height and width of the 2D convolution window : (3,3)
        #activation function : relu
        #input_shape : shape of image matrix : 48x48x1 : (Gray Scale Image)
        model.add(Conv2D(8, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='elu', input_shape=(128,128,3))) 
        #Batch Normalization : It is used to normalize the output of the previous layers. The activations scale the input layer in normalization.
        #Using batch normalization learning becomes efficient also it can be used as regularization to avoid overfitting of the model.  
        model.add(BatchNormalization())

        #--------------------------------Second layer------------------------------------------# 
        model.add(Conv2D(8, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='elu')) 
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2))) #Down-sampling opertaion to reduce dimensionality of feature map
        model.add(Dropout(0.2)) #regularization technique to reduce overfitting: dropouts randomly switches off some neurons in the network which forces the data to find new paths

        #--------------------------------Third layer------------------------------------------# 
        model.add(Conv2D(16, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='elu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))

        #--------------------------------Third layer------------------------------------------# 
        model.add(Conv2D(32, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='elu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        #--------------------------------Fourth layer------------------------------------------# 
        model.add(Conv2D(64, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='elu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))

        #--------------------------------five layer------------------------------------------# 
        model.add(Conv2D(128, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='elu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        #--------------------------------six layer------------------------------------------# 
        model.add(Conv2D(256, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='elu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))

        #--------------------------------six layer------------------------------------------# 
        model.add(Conv2D(512, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='elu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        #--------------------------------six layer------------------------------------------# 
        model.add(Conv2D(1024, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='elu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))


        model.add(Flatten()) #converts 3d vector to 1 d vector : Flatten serves as a connection between the convolution and dense layers.

        #--------------------------------Output layers------------------------------------------# 
        model.add(Dense(512, kernel_initializer='he_uniform', activation='elu')) #Dense layer predict labels
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(512, kernel_initializer='he_uniform', activation='elu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(7, activation='softmax'))        #FINAL PREDICTION
        #Softmax makes the output sum up to 1 so the output can be interpreted as probabilities.
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='RMSprop')
    
        return model

In [7]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [8]:
BS = 32
EPOCHS = 60

In [ ]:
result = []
scores_loss = []
scores_acc = []
k_no = 0
for train_index, test_index in kf.split(x):
    X_Train_ = x[train_index]
    Y_Train = y[train_index]
    X_Test_ = x[test_index]
    Y_Test = y[test_index]

    file_path = r"C:\Users\thanawin\Desktop\Deep-learing\90_10_10\weight\weights_best_"+str(k_no)+".hdf5"
    checkpoint = ModelCheckpoint(file_path, monitor='loss', verbose=0, save_best_only=True, mode='min')
    early = EarlyStopping(monitor="loss", mode="min", patience=8)

    callbacks_list = [checkpoint, early]

    model = create_model()
    hist = model.fit_generator(aug.flow(X_Train_, Y_Train), epochs=EPOCHS,validation_data=(X_Test_, Y_Test), callbacks=callbacks_list, verbose=0)
    # model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, validation_data=(X_Test, Y_Test), verbose=1)
    model.load_weights(file_path)
    result.append(model.predict(X_Test_))
    score = model.evaluate(X_Test_,Y_Test, verbose=0)
    scores_loss.append(score[0])
    scores_acc.append(score[1])
    k_no+=1

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13848\1376269716.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(aug.flow(X_Train_, Y_Train), epochs=EPOCHS,validation_data=(X_Test_, Y_Test), callbacks=callbacks_list, verbose=0)


In [ ]:
print(scores_acc,scores_loss)

In [ ]:
value_min = min(scores_loss)
value_index = scores_loss.index(value_min)
print(value_index)

In [ ]:
model.load_weights(r"C:\Users\thanawin\Desktop\Deep-learing\90_10_10\weight\weights_best_"+str(value_index)+".hdf5")

In [ ]:
best_model = model


In [ ]:
score = best_model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

test_image = X_test[0:1]
print (test_image.shape)

print(best_model.predict(test_image))
print(best_model.predict_classes(test_image))
print(y_test[0:1])

#predict
y_pred = best_model.predict(X_test)

In [ ]:
# visualizing losses and accuracy
%matplotlib inline

train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['accuracy']
val_acc=hist.history['val_accuracy']

epochs = range(len(train_acc))

plt.plot(epochs,train_loss,'r', label='train_loss')
plt.plot(epochs,val_loss,'b', label='val_loss')
plt.title('train_loss vs val_loss')
plt.legend()
plt.figure()

plt.plot(epochs,train_acc,'r', label='train_acc')
plt.plot(epochs,val_acc,'b', label='val_acc')
plt.title('train_acc vs val_acc')
plt.legend()
plt.figure()